In [1]:
import numpy as np
import pandas as pd
from scipy.signal import welch, hilbert
from scipy.stats import entropy
import numpy as np
import pandas as pd
import xgboost as xgb
import neurokit2 as nk
import polars as pl
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             make_scorer, precision_score, recall_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier as DTC
import os, glob
import statistics
import joblib, math
import scipy.signal as signal


In [2]:
p1 = pd.read_csv("C:/Users/eleon/Desktop/SDAP/backend/src/data_brazil/Patient 1/00000714-A5BS17960.csv", usecols=[' HR 1 Frequenz [200 Hz]','EMG MR Frequenz [200 Hz]', 'EMG ML Frequenz [200 Hz]'])
p1.columns = ["ECG", "MR", "ML"]

events_data = pd.read_excel("C:/Users/eleon/Desktop/SDAP/backend/src/data_brazil/Patient 1//Patient_1_episodes.xlsx")

#y = create_labels(events_data, len(p1))

In [5]:
start = 200*80
ecg_80s = p1["ECG"][start*9:start*10]

In [110]:
len(ecg_80s)


16000

In [34]:
sampling_rate = 200
ecg = p1["ECG"]
ecg_clean = nk.ecg_clean(ecg, sampling_rate=sampling_rate)
ecg_peaks = nk.ecg_findpeaks(ecg_clean, sampling_rate=sampling_rate)
info, r_peaks_corrected = nk.signal_fixpeaks(ecg_peaks, sampling_rate=sampling_rate, iterative=False, show=False, method="Kubios")

# Calculate RR intervals
rr_intervals = np.diff(r_peaks_corrected) / sampling_rate * 1000

# Insert fake data point
rr_intervals_adjusted = np.insert(rr_intervals, 0, rr_intervals[0])

# Calculate time axis
time_r_peaks = r_peaks_corrected[1:] / sampling_rate  # Time corresponding to the RR intervals
time_r_peaks_adjusted = np.insert(time_r_peaks, 0, 0)  # Add a time point for the fake interval
print(time_r_peaks[0])

rri_adj_df = pd.DataFrame(data={'RRI': rr_intervals_adjusted, 'RRI_t': time_r_peaks_adjusted})

0.93


In [35]:
rri_adj_df

,RRI,RRI_t
0,430.0,0.000
1,430.0,0.930
2,800.0,1.730
3,800.0,2.530
4,815.0,3.345
...,...,...
31423,830.0,26189.815
31424,840.0,26190.655
31425,855.0,26191.510
31426,850.0,26192.360


In [3]:
def calculate_hrv(ecg_80s):
    ecg_clean = nk.ecg_clean(ecg_80s, sampling_rate=200)
    ecg_peaks = nk.ecg_findpeaks(ecg_clean, sampling_rate=200)
    info, r_peaks_corrected = nk.signal_fixpeaks(ecg_peaks, sampling_rate=200, iterative=False, show=False, method="Kubios")

    hrv = nk.hrv(r_peaks_corrected,sampling_rate=200, show=False)

    mean = hrv["HRV_MeanNN"][0]
    median = hrv["HRV_MedianNN"][0]
    sdnn = hrv["HRV_SDNN"][0]
    min = hrv["HRV_MinNN"][0]
    max = hrv["HRV_MaxNN"][0]
    vlf = hrv["HRV_VLF"][0]
    vhf = hrv["HRV_VHF"][0]
    lf = hrv["HRV_LF"][0]
    hf = hrv["HRV_HF"][0]
    lf_hf = hrv["HRV_LFHF"][0]


    return mean, median, sdnn, min, max, vlf, vhf, lf, hf, lf_hf

In [4]:
import numpy as np
import pandas as pd

# Assuming 'sensor_data' is your dataset and it's sampled at 200 Hz
sampling_rate = 200  # 200 samples per second
window_size_1s = sampling_rate  # 200 samples for 1-second window
window_size_90s = sampling_rate * 90  # 1000 samples for 80-second window
overlap_1s = window_size_1s // 2  # 100 samples for 0.5 second overlap
overlap_90s = window_size_90s // 2  # 9'000 samples for 45 second overlap

event_intervals = events_data[['Time in Sec', 'Event Duration']].values[1:]

event_intervals = [(start, start + duration) for start, duration in event_intervals]


In [9]:
def next_power_of_2(x):
    return 1 if x == 0 else 2 ** (x - 1).bit_length()

def spectrum(signal, sampling_rate):
    m = len(signal)
    n = next_power_of_2(m)
    y = np.fft.fft(signal, n)
    yh = y[0:int(n / 2 - 1)]
    fh = (sampling_rate / n) * np.arange(0, n / 2 - 1, 1)
    power = np.real(yh * np.conj(yh) / n)

    return fh, power


def frequency_ratio(frequency, power):
    power_low = power[(frequency >= 1) & (frequency <= 30)]  # Low range: 1-30 Hz
    power_high = power[(frequency > 30) & (frequency <= 100)]  # High range: 30-100 Hz (up to Nyquist)

    ULC = np.sum(power_low)
    UHC = np.sum(power_high)

    # Avoid division by zero in case the high-frequency power is 0
    if UHC == 0:
        return np.inf  # Return infinity or handle it as appropriate
    else:
        return ULC / UHC

def mean_freq(frequency, power):
    num = 0
    den = 0
    for i in range(int(len(power) / 2)):
        num += frequency[i] * power[i]
        den += power[i]
    
    if den == 0:
        return np.nan
    else:
        return num / den


def median_freq(frequency, power):
    power_total = np.sum(power) / 2
    temp = 0
    tol = 0.01
    errel = 1
    i = 0
    try:
        while abs(errel) > tol:
            temp += power[i]
            errel = (power_total - temp) / power_total
            i += 1
            if errel < 0:
                errel = 0
                i -= 1

        return frequency[i]
    except:
        return np.nan
    
def peak_freq(frequency, power):
    try:
        return frequency[power.argmax()]
    
    except:
        return np.nan


In [61]:
len(p1)

5238800

In [20]:
features_1s = []
labels = []
features_90s = []

ecg_data = p1['ECG'].values
mr_data = p1["MR"].values
ml_data = p1["ML"].values
previous_features_1s = None

for i in range(window_size_1s, len(p1), overlap_1s):

    # Extract the 1-second window
    mr_window = mr_data[i-window_size_1s:i]
    ml_window = ml_data[i-window_size_1s:i]

    # Standard Deviation
    std_mr = np.std(mr_window)
    std_ml = np.std(ml_window)

    # Variance
    var_mr = np.var(mr_window)
    var_ml = np.var(ml_window)

    frequency_mr, power_mr = spectrum(mr_window, sampling_rate)
    frequency_ml, power_ml = spectrum(ml_window, sampling_rate)
    
    # Frequency power
    fr_mr =frequency_ratio(frequency_mr, power_mr) 
    fr_ml =frequency_ratio(frequency_mr, power_mr)

    # Mean power
    mnp_mr = np.sum(power_mr) / len(power_mr)
    mnp_ml = np.sum(power_ml) / len(power_ml)

    
    # Total power
    tot_mr = np.sum(power_mr)
    tot_ml = np.sum(power_ml)

    #Mean Frequency
    mnf_mr = mean_freq(frequency_mr, power_mr)
    mnf_ml = mean_freq(frequency_ml, power_ml)

    # Median frequency
    mdf_mr = median_freq(frequency_mr, power_mr)
    mdf_ml = median_freq(frequency_ml, power_ml)

    # Peak frequency
    pkf_mr = peak_freq(frequency_mr, power_mr)
    pkf_ml = peak_freq(frequency_ml, power_ml)

    start_time = (i-window_size_1s) / sampling_rate
    end_time = ((i-window_size_1s) + window_size_1s) / sampling_rate
    
    current_features = [start_time, end_time, std_mr, std_ml, var_mr, var_ml, fr_mr, fr_ml, mnp_mr, mnp_ml, tot_mr, tot_ml, mnf_mr, mnf_ml, mdf_mr, mdf_ml, pkf_mr, pkf_ml]
    features_1s.append(current_features)

    previous_features_1s = current_features

    # Assign label based on event timing from the Excel file
    current_time = (i-window_size_1s) / sampling_rate  # Convert start index to time in seconds
    label = 0  # Default label is 0 (no event)
    
    # Loop through the events and check if current time falls within any event interval
    for event_start, event_end in event_intervals:
        if event_start <= current_time < event_end:
            label = 1  # Mark as event present
            break  # Stop checking once the window falls within an event

    labels.append(label)

    if i % 200000 == 0:
        print(f"i: {i}")

# Step 4: Second loop for 90-second ECG window and HRV calculation
for i in range(window_size_90s, len(ecg_data), overlap_90s):
    # Extract 90 seconds of ECG data
    window_90s = ecg_data[i - window_size_90s:i]  # 90-second window

    # Step 5: Calculate HRV metrics for 90-second window
    mean, median, sdnn, min, max, vlf, vhf, lf, hf, lf_hf = calculate_hrv(window_90s)

    # Step 6: Assign HRV metrics to each 1-second window in this 90-second period
    num_1s_windows_in_90s = window_size_90s // window_size_1s
    for _ in range(num_1s_windows_in_90s):
        features_90s.append([mean, median, sdnn, min, max, vlf, vhf, lf, hf, lf_hf])


combined_features = [f1 + f28 for f1, f28 in zip(features_1s, features_90s[:len(features_1s)])]

i: 200000
i: 400000
i: 600000
i: 800000
i: 1000000
i: 1200000
i: 1400000
i: 1600000
i: 1800000
i: 2000000
i: 2200000
i: 2400000
i: 2600000
i: 2800000
i: 3000000
i: 3200000
i: 3400000
i: 3600000
i: 3800000
i: 4000000
i: 4200000
i: 4400000
i: 4600000
i: 4800000
i: 5000000
i: 5200000


c:\Python311\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
c:\Python311\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
c:\Python311\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for mor

In [21]:
columns = ["start_time", "end_time", "std_mr", "std_ml", "var_mr", "var_ml",
            "fr_mr", "fr_ml", "mnp_mr", "mnp_ml", "tot_mr", "tot_ml", "mnf_mr",
            "mnf_ml", "mdf_mr", "mdf_ml", "pkf_mr", "pkf_ml", "HRV_mean", "HRV_median", "HRV_sdnn", "HRV_min", 
            "HRV_max", "HRV_vlf", "HRV_vhf", "HRV_lf", "HRV_hf", "HRV_lf_hf"]

In [22]:
features = pd.DataFrame(combined_features, columns=columns)

In [24]:
features

,start_time,end_time,std_mr,std_ml,var_mr,var_ml,fr_mr,fr_ml,mnp_mr,mnp_ml,...,HRV_mean,HRV_median,HRV_sdnn,HRV_min,HRV_max,HRV_vlf,HRV_vhf,HRV_lf,HRV_hf,HRV_lf_hf
0,0.0,1.0,14.870502,27.859298,221.131823,776.140493,0.053870,0.053870,174.121290,611.100247,...,789.292035,795.0,38.706008,430.0,825.0,NaN,0.002458,0.045247,0.003658,12.370770
1,0.5,1.5,11.799874,25.761106,139.237015,663.634589,0.080222,0.080222,109.052928,521.954821,...,789.292035,795.0,38.706008,430.0,825.0,NaN,0.002458,0.045247,0.003658,12.370770
2,1.0,2.0,2.998260,18.270751,8.989564,333.820339,0.831630,0.831630,7.073906,262.766780,...,789.292035,795.0,38.706008,430.0,825.0,NaN,0.002458,0.045247,0.003658,12.370770
3,1.5,2.5,3.103752,18.197867,9.633277,331.162351,0.617677,0.617677,7.567821,260.568100,...,789.292035,795.0,38.706008,430.0,825.0,NaN,0.002458,0.045247,0.003658,12.370770
4,2.0,3.0,3.478529,18.604113,12.100166,346.113032,0.992752,0.992752,9.522304,272.358739,...,789.292035,795.0,38.706008,430.0,825.0,NaN,0.002458,0.045247,0.003658,12.370770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52285,26142.5,26143.5,3.292378,2.237865,10.839755,5.008042,0.556648,0.556648,8.533901,3.942424,...,810.636364,830.0,41.237225,715.0,870.0,NaN,0.000112,0.014328,0.001455,9.850379
52286,26143.0,26144.0,2.716582,2.157030,7.379816,4.652779,0.899591,0.899591,5.809787,3.662457,...,810.636364,830.0,41.237225,715.0,870.0,NaN,0.000112,0.014328,0.001455,9.850379
52287,26143.5,26144.5,2.904713,2.312253,8.437357,5.346514,1.046712,1.046712,6.643723,4.208211,...,810.636364,830.0,41.237225,715.0,870.0,NaN,0.000112,0.014328,0.001455,9.850379
52288,26144.0,26145.0,6.608897,4.358165,43.677526,18.993604,0.420970,0.420970,34.383163,14.955559,...,810.636364,830.0,41.237225,715.0,870.0,NaN,0.000112,0.014328,0.001455,9.850379


In [15]:
len(p1)/2000

2619.4

In [42]:
features = pd.DataFrame(combined_features, columns=columns)

In [36]:
rri_adj_df

,RRI,RRI_t
0,430.0,0.000
1,430.0,0.930
2,800.0,1.730
3,800.0,2.530
4,815.0,3.345
...,...,...
31423,830.0,26189.815
31424,840.0,26190.655
31425,855.0,26191.510
31426,850.0,26192.360


In [17]:
if len(labels) != len(features):
    difference = len(labels) - len(features)



for i in range(difference):
    del labels[-1]

In [18]:
len(labels)

52240

In [19]:
features["y"] = labels

In [21]:
features[features["y"] == 1]

,MEAN,STD,MIN,MAX,HR,SDNN,LFHF,y
5650,13.824431,ECG 179.447095 MR 12.677967 ML 8...,-562.194434,1201.246428,calculate_heart_rate(window_5s),26.705675,8.931327,1
5651,3.905713,ECG 236.541656 MR 24.630690 ML 17...,-588.000961,1308.773235,calculate_heart_rate(window_5s),26.705675,8.931327,1
5972,5.013883,ECG 260.844163 MR 12.202585 ML 8...,-575.114821,1519.508918,calculate_heart_rate(window_5s),22.596726,4.506074,1
5973,0.140225,ECG 188.578236 MR 11.284283 ML 7...,-575.114821,1420.584106,calculate_heart_rate(window_5s),22.596726,4.506074,1
5974,10.675077,ECG 180.357551 MR 13.353067 ML 8...,-545.007374,1248.541015,calculate_heart_rate(window_5s),22.596726,4.506074,1
...,...,...,...,...,...,...,...,...
51815,10.845224,ECG 162.028685 MR 8.858761 ML 7...,-366.800315,704.167519,calculate_heart_rate(window_5s),81.075673,13.402375,1
51816,-15.203065,ECG 122.725257 MR 6.798423 ML 4...,-366.800315,631.049150,calculate_heart_rate(window_5s),81.075673,13.402375,1
51817,-26.362711,ECG 127.405267 MR 9.770600 ML 5...,-500.133736,514.920030,calculate_heart_rate(window_5s),81.075673,13.402375,1
51818,-50.047993,ECG 116.825924 MR 14.256071 ML 8...,-500.133736,514.920030,calculate_heart_rate(window_5s),81.075673,13.402375,1
